In [1]:
# !pip install -r requirements.txt 

In [2]:
# !wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml<br>
# !sudo docker-compose up -d

In [3]:
from pymilvus import connections
import psycopg2

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from flask import Flask, request
from flask_cors import CORS, cross_origin

from OpenSSL import SSL



/home/milan_v/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import textract
# import urllib.request
import requests
import re
import os
from collections import Counter
import operator

import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

---

## Database Connections & Global Variables

In [28]:
global switch
switch = True

In [6]:
def connectdb():
    
    connections.connect(host='104.196.170.44', port='19530')
    global conn 
    conn = psycopg2.connect(database="alpha_alberta", host='35.203.122.7', port='5432', user='my_alpha_user', password='alpha_alberta@') # postgres
    print("CONNECTION ====",conn)

    global TABLE_NAME, TABLE_NAME_, field_name
    global TABLE_NAME_1, TABLE_NAME_1_, field_name_1
    global TABLE_NAME_2, TABLE_NAME_2_, field_name_2
    global TABLE_NAME_3, TABLE_NAME_3_, field_name_3
    global TABLE_NAME_4, TABLE_NAME_4_, field_name_4
    global TABLE_NAME_5, TABLE_NAME_5_, field_name_5
    global TABLE_NAME_6, TABLE_NAME_6_, field_name_6
    
    TABLE_NAME = "Skills_Embd"
    TABLE_NAME_ = "Skills_Embd_"
    field_name = "Skills"

    TABLE_NAME_1 = "Skills_Embd"
    TABLE_NAME_1_ = "Skills_Embd_"
    field_name_1 = "Skills"

    TABLE_NAME_2 = "Embds"
    TABLE_NAME_2_ = "Embds_"
    field_name_2 = "Mix_Fields"

    TABLE_NAME_3 = "Biz_Embds"
    TABLE_NAME_3_ = "Biz_Embds_"
    field_name_3 = "Biz"

    TABLE_NAME_4 = "Ppl_Embds"
    TABLE_NAME_4_ = "Ppl_Embds_"
    field_name_4 = "Ppl"

    TABLE_NAME_5 = "cmt_Embds"
    TABLE_NAME_5_ = "cmt_Embds_"
    field_name_5 = "cmt"

    TABLE_NAME_6 = "doc_Embds"
    TABLE_NAME_6_ = "doc_Embds_"
    field_name_6 = "doc"

connectdb()

CONNECTION ==== <connection object at 0x7f12d7edcb80; dsn: 'user=my_alpha_user password=xxx dbname=alpha_alberta host=35.203.122.7 port=5432', closed: 0>


In [7]:
# paraphrase-mpnet-base-v2 
# multi-qa-mpnet-base-dot-v1
# all-with_prefix-t5-base-v1
# all-mpnet-base-v2
# all-MiniLM-L6-v2
# paraphrase-MiniLM-L6-v2 **BEST**
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
cursor = conn.cursor()

search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

In [8]:
def preprocess(text):   

    text = text.strip('\n')
    text = text.strip('\t')
    text = text.strip('\r')

    text = re.sub("\s+" , " ", text)
    text = text.strip()
    text = text.lower()
    
    return text

# preprocess('   milan LEO ')

## Column Names

In [10]:
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())

[('django_migrations',), ('django_content_type',), ('api_user',), ('api_professionalexperience',), ('api_personallabequipments',), ('api_personaleducation',), ('api_personaldocumentupload',), ('api_personalinformation',), ('api_businessinformation',), ('api_personalcertificates',), ('api_languagesspoken',), ('api_inviteduser',), ('api_honorsandawards',), ('api_helpcontactus',), ('api_businesslabequipments',), ('api_businessdocumentupload',), ('api_user_role',), ('api_userrole',), ('auth_permission',), ('django_admin_log',), ('api_businesslocation',), ('auth_group',), ('auth_group_permissions',), ('challenge_creator_comment',), ('challenge_creator_challengestatement',), ('challenge_creator_challengestatement_industry',), ('challenge_creator_industry',), ('challenge_creator_challengeclosereview',), ('lab_profile_labprofile',), ('lab_profile_lablocation',), ('lab_profile_labequipment',), ('lab_profile_labdocumentupload',), ('django_session',), ('solution_seeker_solutioninformationprofile'

In [12]:
print('challenge_creator_challengestatement:')
print('-'*50)
cursor.execute("Select * FROM challenge_creator_challengestatement LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_user:')
print('-'*50)
cursor.execute("Select * FROM api_user LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_personalinformation:')
print('-'*50)
cursor.execute("Select * FROM api_personalinformation LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_personallabequipments:')
print('-'*50)
cursor.execute("Select * FROM api_personallabequipments LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_businessinformation:')
print('-'*50)
cursor.execute("Select * FROM api_businessinformation LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_businesslabequipments:')
print('-'*50)
cursor.execute("Select * FROM api_businesslabequipments LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_businesslocation:')
print('-'*50)
cursor.execute("Select * FROM api_businesslocation LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('challenge_creator_comment:')
print('-'*50)
cursor.execute("Select * FROM challenge_creator_comment LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_personaldocumentupload:')
print('-'*50)
cursor.execute("Select * FROM api_personaldocumentupload LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('api_businessdocumentupload:')
print('-'*50)
cursor.execute("Select * FROM api_businessdocumentupload LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('lab_profile_labdocumentupload:')
print('-'*50)
cursor.execute("Select * FROM lab_profile_labdocumentupload LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

print('='*50)

print('lab_profile_labprofile:')
print('-'*50)
cursor.execute("Select * FROM lab_profile_labprofile LIMIT 0") 
colnames = [desc[0] for desc in cursor.description]
print(colnames)

# api_personaldocumentupload api_businessdocumentupload lab_profile_labdocumentupload

challenge_creator_challengestatement:
--------------------------------------------------
['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']
api_user:
--------------------------------------------------
['id', 'password', 'last_login', 'uid', 'email', 'username', 'can_invite_others', 'is_fresh_login', 'date_joined', 'is_active', 'is_admin', 'is_challenge_creator', 'is_solution_provider', 'is_email_verified', 'is_manager', 'is_staff', 'is_agreed_terms_and_conditions', 'created_at', 'modified_at']
api_personalinformation:
--------------------------------------------------
['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 'bio', 'office_phone', 'personal_phone', 'address_line_1', 'add

---

## Recommendations

In [11]:
def update_rcmd():
    
    if switch == True:
        TN = TABLE_NAME_
    else:
        TN = TABLE_NAME
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    skills = FieldSchema(name = field_name, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, skills], description = "example collection")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name, index_params=index_param)
    
#   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
#    'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
#    'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
#    'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']
    
    try:
        cursor.execute("Select id, skills from challenge_creator_challengestatement")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_rcmd')
        connectdb()
    
    lst_pid, lst_skills = [], []
    for i, j in rows_pos:
        if j == None:
            continue
        lst_pid.append(i)
        lst_skills.append(preprocess(j))
    
    if len(lst_skills) != 0:
        sentence_embeddings = model.encode(lst_skills)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, em])
        print('RCMD updated on', TN)
    else:
        print("There is no Data")

In [12]:
# update_rcmd()

In [13]:
def query_rcmd(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
#     print('\n'*5)
#     print(query_embeddings)
#     print('\n'*5)
    
    if switch == True:
        collection = Collection(TABLE_NAME)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_)
        collection.load()
        
    results = collection.search(query_embeddings, field_name, param=search_params, limit=1000, expr=None)
    
    recommendation1, recommendation2 = [], []
    for result in results[0]:
#             print('-'*36)
#             print('\n')
#             print("RESULT -->", result.id)
#             print('-'*36)
#             print('\n')
#         print(result.distance)
#         if result.distance < 1.50:
    
#           ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
#            'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
#            'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
#            'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']       
            
            try:
                sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows = cursor.fetchall()
            except:
                print('query at query_rcmd 1 ==>', result.id)
                connectdb()
                
            if len(rows) != 0:
                
#                 print('-'*36)
#                 print('\n')
#                 print("USER_ID -->", rows[0][-1])
#                 print("COMPANY_ID -->", rows[0][-2])
#                 print('-'*36)
#                 print('\n')
                
                if rows[0][11] == True:
                    
                    if rows[0][-1] != None and rows[0][-1] != '':
                        try:
                            get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                            cursor.execute(get_user_name)
                            fetch_user_name = cursor.fetchall()
                            fetch_user_name = fetch_user_name[0][0]
                        except:
                            print('query at query_rcmd 2 ==>', rows[0][-1])
                            connectdb()
                        
                        try:
                            get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                            cursor.execute(get_profile_picture)
                            fetch_profile_picture = cursor.fetchall()

                            if len(fetch_profile_picture) != 0:
                                fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                            else:
                                fetch_profile_picture = ''
                        except:
                            print('query at query_rcmd 3 ==>', rows[0][-1])
                            connectdb()
                    
                    else:
                        fetch_user_name = None
                        fetch_profile_picture = ''

                    if rows[0][-2] != None and rows[0][-2] != '':

                        try:
                            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                            cursor.execute(get_company_name)
                            fetch_company_name = cursor.fetchall()
                            fetch_company_name = fetch_company_name[0][0]
                        except:
                            print('query at query_rcmd 4 ==>', rows[0][-2])
                            connectdb()

                    else:
                        fetch_company_name = None

                    recommendation1.append({'id' : rows[0][0], 
                                           'challenge_title': rows[0][1],
                                           'challenge_description': rows[0][2],
                                           'challenge_location': rows[0][3],
                                           'challenge_state': rows[0][4],
                                           'challenge_city': rows[0][5],
                                           'skills': rows[0][6],
                                           'status_type': rows[0][7],
                                           'post_type': rows[0][8],
                                           'is_active': rows[0][9],
                                           'is_archieve': rows[0][10],
                                           'is_student_required': rows[0][11],
                                           'created_at': rows[0][12],
                                           'challenge_end_date': rows[0][13],
                                           'updated_at': rows[0][14],
                                           'created_at_time': rows[0][15],
                                           'updated_at_time': rows[0][16],
                                           'company_name_id': rows[0][17],
                                           'user_id': rows[0][18],
                                           'user_name': fetch_user_name,
                                           'company_name': fetch_company_name,
                                           'profile_picture': fetch_profile_picture,
                                           'distance': result.distance})


                if rows[0][11] == False:
                    
                    if rows[0][-1] != None and rows[0][-1] != '':

                        try:
                            get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                            cursor.execute(get_user_name)
                            fetch_user_name = cursor.fetchall()
                            fetch_user_name = fetch_user_name[0][0]
                        except:
                            print('query at query_rcmd 5 ==>', rows[0][-1])
                            connectdb()

                        try:
                            get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                            cursor.execute(get_profile_picture)
                            fetch_profile_picture = cursor.fetchall()

                            if len(fetch_profile_picture) != 0:
                                fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                            else:
                                fetch_profile_picture = ''
                        except:
                            print('query at query_rcmd 6 ==>', rows[0][-1])
                            connectdb()
                            
                    else:
                        fetch_user_name = None
                        fetch_profile_picture = ''

                    if rows[0][-2] != None and rows[0][-2] != '':
                        try:
                            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                            cursor.execute(get_company_name)
                            fetch_company_name = cursor.fetchall()
                            fetch_company_name = fetch_company_name[0][0]
                        except:
                            print('query at query_rcmd 7 ==>', rows[0][-2])
                            connectdb()
                    else:
                        fetch_company_name = None

                    recommendation2.append({'id' : rows[0][0], 
                                           'challenge_title': rows[0][1],
                                           'challenge_description': rows[0][2],
                                           'challenge_location': rows[0][3],
                                           'challenge_state': rows[0][4],
                                           'challenge_city': rows[0][5],
                                           'skills': rows[0][6],
                                           'status_type': rows[0][7],
                                           'post_type': rows[0][8],
                                           'is_active': rows[0][9],
                                           'is_archieve': rows[0][10],
                                           'is_student_required': rows[0][11],
                                           'created_at': rows[0][12],
                                           'challenge_end_date': rows[0][13],
                                           'updated_at': rows[0][14],
                                           'created_at_time': rows[0][15],
                                           'updated_at_time': rows[0][16],
                                           'company_name_id': rows[0][17],
                                           'user_id': rows[0][18],
                                           'user_name': fetch_user_name,
                                           'company_name': fetch_company_name,
                                           'profile_picture': fetch_profile_picture,
                                           'distance': result.distance})
            
#             print('='*50)
#             print('\n')
            
    recommendation = recommendation2 + recommendation1

    return recommendation

In [14]:
# print(1, '-'*25, '>')
# rcmd = query_rcmd('vuejs')
# for i, j in enumerate(rcmd):
    
#     print(i, "-->")
#     print('Distance:', j['distance'])
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)

# #     if i == 1:
# #         break

1 ------------------------- >
0 -->
Distance: 1.3457834720611572
id : 17
challenge_title : Draft
challenge_description : Draft
challenge_location : Canada
challenge_state : Alberta
challenge_city : Alberta
skills : 
status_type : New
post_type : Draft
is_active : True
is_archieve : False
is_student_required : False
created_at : 2022-12-07
challenge_end_date : None
updated_at : 2022-12-07
created_at_time : 2022-12-07 08:42:41.410626+00:00
updated_at_time : 2022-12-07 08:42:41.410628+00:00
company_name_id : None
user_id : 55
user_name : raj.p
company_name : None
profile_picture : /dev_asp_bucket/userprofile/Rajkumar_Popat_.jpg
distance : 1.3457834720611572
1 -->
Distance: 1.3457834720611572
id : 18
challenge_title : Draft Title 2
challenge_description : Nature of challenge
challenge_location : Canada
challenge_state : Alberta
challenge_city : City
skills : 
status_type : New
post_type : Draft
is_active : True
is_archieve : False
is_student_required : False
created_at : 2022-12-07
challen

In [15]:
# import psutil

# hdd = psutil.disk_usage('/')

# print ("Total GiB:", hdd.total / (10**9))
# print ("Used GiB:", hdd.used / (10**9))
# print ("Free GiB:", hdd.free / (10**9))

In [16]:
# v = psutil.virtual_memory()
# print(v)
# total = v.total >> 30
# print(total)

In [17]:
# 2112405504 >> 30

In [20]:
# try:
# sql = "select * from challenge_creator_challengestatement where id = 40;"
# cursor.execute(sql)
# rows = cursor.fetchall()
# print(rows)
# except:
# print('query at query_rcmd 1')
# connectdb()
    
# print(rows[0][-1])
# print(rows[0][-2])

# try:
#     get_user_name = "select username from api_user where id =" + str(rows[0][-2]) + ";"
#     cursor.execute(get_user_name)
#     fetch_user_name = cursor.fetchall()
#     fetch_user_name = fetch_user_name[0][0]
#     print(fetch_user_name)
# except:
#     print('query at query_rcmd 5')
#     connectdb()
    
# try:
# get_company_name = "select company_name from api_businessinformation where id = 7;"
# cursor.execute(get_user_name)
# fetch_company_name = cursor.fetchall()
# fetch_company_name = fetch_company_name[0][0]
# except:
#     print('query at query_rcmd 4 ==>')
#     connectdb()

# get_user_name = "select username from api_user where id = 67;"
# cursor.execute(get_user_name)
# fetch_user_name = cursor.fetchall()
# print(fetch_user_name)
# fetch_user_name = fetch_user_name[0][0]

In [21]:
# connectdb()

## Student Recommendations

In [82]:
def update_stdrcmd():
    
    if switch == True:
        TN = TABLE_NAME_1_
    else:
        TN = TABLE_NAME_1
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    skills = FieldSchema(name = field_name_1, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, skills], description = "example collection")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_1, index_params=index_param)
    
#   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
#    'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
#    'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
#    'created_at_time', 'updated_at_time', 'company_name_id', 'user_id'] 
    
    try:
        cursor.execute("Select id, skills from challenge_creator_challengestatement")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_stdrcmd')
        connectdb()
    
    lst_pid, lst_skills = [], []
    for i, j in rows_pos:
        if j == None:
            continue
        lst_pid.append(i)
        lst_skills.append(preprocess(j))
    
    if len(lst_skills) != 0:
        sentence_embeddings = model.encode(lst_skills)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, em])
        print('STDRCMD updated on', TN)
    else:
        print("There is no Data")

In [83]:
# update_stdrcmd()

STDRCMD updated on Skills_Embd_


In [24]:
def query_stdrcmd(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_1)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_1_)
        collection.load()
        
    results = collection.search(query_embeddings, field_name_1, param=search_params, limit=1000, expr=None)
    
    recommendation1, recommendation2 = [], []
    for result in results[0]:
        
#         if result.distance < 1.50:
    
    #       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #        'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #        'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #        'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']       
            
            try:
                sql = "select * from challenge_creator_challengestatement AS ccc where id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows = cursor.fetchall()
            except:
                print('query at query_stdrcmd 1')
                connectdb()
            
            if len(rows) != 0:
                
                if rows[0][11] == True:

                    try:
                        get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                        cursor.execute(get_user_name)
                        fetch_user_name = cursor.fetchall()
                        fetch_user_name = fetch_user_name[0][0]
                    except:
                        print('query at query_stdrcmd 2')
                        connectdb()
                        
                    try:
                        get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                        cursor.execute(get_profile_picture)
                        fetch_profile_picture = cursor.fetchall()

                        if len(fetch_profile_picture) != 0:
                            fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                        else:
                            fetch_profile_picture = ''
                    except:
                        print('query at query_stdrcmd 3')
                        connectdb()

                    if rows[0][-2] is not None:

                        try:
                            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                            cursor.execute(get_company_name)
                            fetch_company_name = cursor.fetchall()
                            fetch_company_name = fetch_company_name[0][0]
                        except:
                            print('query at query_stdrcmd 4')
                            connectdb()

                    else:
                        fetch_company_name = None

                    recommendation1.append({'id' : rows[0][0], 
                                           'challenge_title': rows[0][1],
                                           'challenge_description': rows[0][2],
                                           'challenge_location': rows[0][3],
                                           'challenge_state': rows[0][4],
                                           'challenge_city': rows[0][5],
                                           'skills': rows[0][6],
                                           'status_type': rows[0][7],
                                           'post_type': rows[0][8],
                                           'is_active': rows[0][9],
                                           'is_archieve': rows[0][10],
                                           'is_student_required': rows[0][11],
                                           'created_at': rows[0][12],
                                           'challenge_end_date': rows[0][13],
                                           'updated_at': rows[0][14],
                                           'created_at_time': rows[0][15],
                                           'updated_at_time': rows[0][16],
                                           'company_name_id': rows[0][17],
                                           'user_id': rows[0][18],
                                           'user_name': fetch_user_name,
                                           'company_name': fetch_company_name,
                                           'profile_picture': fetch_profile_picture})


                if rows[0][11] == False:

                    try:
                        get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                        cursor.execute(get_user_name)
                        fetch_user_name = cursor.fetchall()
                        fetch_user_name = fetch_user_name[0][0]
                    except:
                        print('query at query_stdrcmd 5')
                        connectdb()
                        
                    try:
                        get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                        cursor.execute(get_profile_picture)
                        fetch_profile_picture = cursor.fetchall()

                        if len(fetch_profile_picture) != 0:
                            fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                        else:
                            fetch_profile_picture = ''
                    except:
                        print('query at query_stdrcmd 6')
                        connectdb()

                    if rows[0][-2] is not None:
                        try:
                            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                            cursor.execute(get_company_name)
                            fetch_company_name = cursor.fetchall()
                            fetch_company_name = fetch_company_name[0][0]
                        except:
                            print('query at query_stdrcmd 7')
                            connectdb()
                    else:
                        fetch_company_name = None

                    recommendation2.append({'id' : rows[0][0], 
                                           'challenge_title': rows[0][1],
                                           'challenge_description': rows[0][2],
                                           'challenge_location': rows[0][3],
                                           'challenge_state': rows[0][4],
                                           'challenge_city': rows[0][5],
                                           'skills': rows[0][6],
                                           'status_type': rows[0][7],
                                           'post_type': rows[0][8],
                                           'is_active': rows[0][9],
                                           'is_archieve': rows[0][10],
                                           'is_student_required': rows[0][11],
                                           'created_at': rows[0][12],
                                           'challenge_end_date': rows[0][13],
                                           'updated_at': rows[0][14],
                                           'created_at_time': rows[0][15],
                                           'updated_at_time': rows[0][16],
                                           'company_name_id': rows[0][17],
                                           'user_id': rows[0][18],
                                           'user_name': fetch_user_name,
                                           'company_name': fetch_company_name,
                                           'profile_picture': fetch_profile_picture})
                
    recommendation = recommendation1 + recommendation2

    return recommendation

In [25]:
# rcmd = query_stdrcmd('django react html')
# for i, j in enumerate(rcmd):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)

#     if i == 1:
#         break

---

## Search

In [90]:
def update_search():
    
    if switch == True:
        TN = TABLE_NAME_2_
    else:
        TN = TABLE_NAME_2
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    embds = FieldSchema(name = field_name_2, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, embds], description = "This table contails embeddings \
                              of mixed fields i.e. id, challenge_title, challenge_description, \
                              challenge_location and skills")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_2, index_params=index_param)
    
#   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
#    'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
#    'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
#    'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']     
    
    try:
        cursor.execute("Select id, challenge_title, skills, challenge_location, challenge_state, challenge_city, \
                        challenge_description from challenge_creator_challengestatement")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_search')
        connectdb()
    
    
    lst_pid, lst_mix = [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])
        
        text = ''
        for m in i[1:]:
            if m is not None:
                text = text + ' ' + str(m)
                
        lst_mix.append(preprocess(text))
    
    if len(lst_mix) != 0:
        sentence_embeddings = model.encode(lst_mix)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, em])
        print('SEARCH updated on', TN)
    else:
        print("There is no Data")

In [92]:
# update_search()

SEARCH updated on Embds_


In [28]:
def query_search(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_2)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_2_)
        collection.load()
    
    results = collection.search(query_embeddings, field_name_2, param=search_params, limit=1000, expr=None)
    
    search = []
    for result in results[0]:
        
#         if result.distance < 1.8:
        
    #       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #        'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #        'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #        'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']   
            
            try:
                sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows = cursor.fetchall()
            except:
                print('query at query_search 1')
                connectdb()
                
            try:
                get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                cursor.execute(get_profile_picture)
                fetch_profile_picture = cursor.fetchall()

                if len(fetch_profile_picture) != 0:
                    fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                else:
                    fetch_profile_picture = ''
            except:
                print('query at query_search 2')
                connectdb()
            
            if len(rows) != 0:
                
                try:
                    get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                    cursor.execute(get_user_name)
                    fetch_user_name = cursor.fetchall()
                    fetch_user_name = fetch_user_name[0][0]
                except:
                    print('query at query_search 3')
                    connectdb()

                if rows[0][-2] is not None:

                    try:
                        get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                        cursor.execute(get_user_name)
                        fetch_company_name = cursor.fetchall()
                        fetch_company_name = fetch_company_name[0][0]
                    except:
                        print('query at query_search 3')
                        connectdb()

                else:
                    fetch_company_name = None
                    
                search.append({'id' : rows[0][0], 
                               'challenge_title': rows[0][1],
                               'challenge_description': rows[0][2],
                               'challenge_location': rows[0][3],
                               'challenge_state': rows[0][4],
                               'challenge_city': rows[0][5],
                               'skills': rows[0][6],
                               'status_type': rows[0][7],
                               'post_type': rows[0][8],
                               'is_active': rows[0][9],
                               'is_archieve': rows[0][10],
                               'is_student_required': rows[0][11],
                               'created_at': rows[0][12],
                               'challenge_end_date': rows[0][13],
                               'updated_at': rows[0][14],
                               'created_at_time': rows[0][15],
                               'updated_at_time': rows[0][16],
                               'company_name_id': rows[0][17],
                               'user_id': rows[0][18],
                               'user_name': fetch_user_name,
                               'company_name': fetch_company_name,
                               'profile_picture': fetch_profile_picture})
    
    return search

In [29]:
# search = query_search('database')
# for i, j in enumerate(search):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
# #     if i == 1:
# #         break

---

## Business

In [13]:
def update_biz():
    
    if switch == True:
        TN = TABLE_NAME_3_
    else:
        TN = TABLE_NAME_3
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    biz = FieldSchema(name = field_name_3, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, biz], description = "example collection")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_3, index_params=index_param)
    
#   ['id', 'userid', 'company_name', 'company_website', 'company_description', 'company_email', 
#    'company_phone', 'company_classification', 'company_type', 'gross_revenue', 'invite_through_company', 
#    'years_in_business', 'start_date', 'end_date', 'created_at', 'modified_at', 'user_id', 'company_headcount']

#   ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
#    'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']
    
    try: # company_classification
        cursor.execute("Select id, company_name, company_description, company_type, \
                        company_website, company_email, company_phone from api_businessinformation")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_biz 1')
        connectdb()
    
    lst_pid, lst_biz = [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])
        
        s = ''
        for m in i[1:]:
            if m is not None:
                s = s + ' ' + str(m)
        
        try:
            cursor.execute("Select company_address_line_1, company_address_line_2, company_city, \
                            company_state, company_country, zip_code from api_businesslocation \
                            WHERE business_id = {};".format(i[0]))
            rows_loc = cursor.fetchall()
        except:
            print('query at update_biz 2')
            connectdb()
        
        for j in rows_loc:
            for n in j:
                if n is not None:
                    s = s + ' ' + str(n)
        
        lst_biz.append(preprocess(s))
    
    if len(lst_biz) != 0:
        sentence_embeddings = model.encode(lst_biz)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, em])
        print('BIZ updated on', TN)
    else:
        print("There is no Data")

In [29]:
# update_biz()

BIZ updated on Biz_Embds_


In [30]:
def query_biz(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_3)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_3_)
        collection.load()
    
    results = collection.search(query_embeddings, field_name_3, param=search_params, limit=1000, expr=None)
    
    biz = []
    for result in results[0]:
        
#         if result.distance < 1.5:

#           ['id', 'userid', 'company_name', 'company_website', 'company_description', 'company_email', 
#            'company_phone', 'company_type', 'gross_revenue', 'invite_through_company', 'years_in_business', 
#            'company_headcount', 'start_date', 'end_date', 'created_at', 'modified_at', 'user_id']
        
#           ['id', 'userid', 'company_name', 'company_website', 'company_description', 'company_email', 
#            'company_phone', 'company_classification', 'company_type', 'gross_revenue', 'invite_through_company', 
#            'years_in_business', 'start_date', 'end_date', 'created_at', 'modified_at', 'user_id', 'company_headcount']

#           ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
#            'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']
            
            try:
                sql = "select * from api_businessinformation where id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows = cursor.fetchall()
            except:
                print('query at query_biz 1')
                connectdb()
            
            try:
                sql = "select * from api_businesslocation where business_id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows_loc = cursor.fetchall()
            except:
                print('query at query_biz 2')
                connectdb()

            loc = []
            if len(rows_loc) != 0:
                for i in rows_loc:
                    loc.append({'id': i[0],
                                'company_address_line_1': i[1],
                                'company_address_line_2': i[2],
                                'company_city': i[3],
                                'company_state': i[4],
                                'company_country': i[5],
                                'zip_code': i[6],
                                'created_at': i[7],
                                'modified_at': i[8],
                                'business_id': i[9],
                                'user_id': i[10]})

            if len(rows) != 0:
                biz.append({'id' : rows[0][0], 
                            'userid': rows[0][1],
                            'company_name': rows[0][2],
                            'company_website': rows[0][3],
                            'company_description': rows[0][4],
                            'company_email': rows[0][5],
                            'company_phone': rows[0][6],
#                             'company_classification': rows[0][7],
                            'company_type': rows[0][7],
                            'gross_revenue': rows[0][8],
                            'invite_through_company': rows[0][9],
                            'years_in_business': rows[0][10],
                            'company_headcount': rows[0][11],
                            'start_date': rows[0][12],
                            'end_date': rows[0][13],
                            'created_at': rows[0][14],
                            'modified_at': rows[0][15],
                            'user_id': rows[0][16],
                            'locations': loc})

    return biz

In [31]:
# biz = query_biz('Kolkata')
# for i, j in enumerate(biz):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
        
#         if m == 'locations':
            
#             print(m, ':')
            
#             for k, l in enumerate(n):
                
#                 print('        ', k, "-->")
#                 print('        ', '='*25)
                
#                 for x, y in l.items():
#                     print('        ', x, ':', y)
                
#         else:
#             print(m, ':', n)
    
#     if i == 1:
#         break

0 -->
id : 7
userid : 2
company_name : Memoranda
company_website : https://Trapo.in
company_description : nothifnf
company_email : manishshaw776@gmail.com
company_phone : 9983479374
company_type : Non Profit
gross_revenue : 78
invite_through_company : True
years_in_business : 22
company_headcount : 3400
start_date : 2022-01-01 00:00:00+00:00
end_date : 2022-12-02 13:44:28+00:00
created_at : 2022-12-02 13:44:28+00:00
modified_at : 2022-12-19 12:40:21.067328+00:00
user_id : 2
locations :
         0 -->
         id : 5
         company_address_line_1 : fjejeej
         company_address_line_2 : fjejwe
         company_city : kolkata
         company_state : Andaman and Nicobar Islands
         company_country : India
         zip_code : 700039
         created_at : False
         modified_at : 2022-12-19 07:29:34.345865+00:00
         business_id : 2022-12-19 07:29:34.346439+00:00
         user_id : 7
         1 -->
         id : 3
         company_address_line_1 : rerkwrwk
         compan

---

## People

In [34]:
def update_ppl():
    
    if switch == True:
        TN = TABLE_NAME_4_
    else:
        TN = TABLE_NAME_4
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    ppl = FieldSchema(name = field_name_4, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, ppl], description = "example collection")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_4, index_params=index_param)
    
#   ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 
#    'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 'profile_picture', 
#    'city', 'state', 'country', 'zip_code', 'is_student', 'created_at', 'modified_at', 'user_id']
    
    try:
        cursor.execute("Select id, first_name, last_name, personal_skills, job_title, \
                        headline, bio, personal_phone, address_line_1, address_line_2, \
                        city, state, country, personal_email, office_phone, zip_code from api_personalinformation")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_ppl')
        connectdb()
    
    lst_pid, lst_ppl = [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])
        
        s = ''
        for m in i[1:]:

            if m is not None:
                s = s + ' ' + str(m)
        
        lst_ppl.append(preprocess(s))
    
    if len(lst_ppl) != 0:
        sentence_embeddings = model.encode(lst_ppl)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, em])
        print('PPL updated on', TN)
    else:
        print("There is no Data")

In [35]:
# update_ppl()

In [36]:
def query_ppl(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_4)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_4_)
        collection.load()
    
    results = collection.search(query_embeddings, field_name_4, param=search_params, limit=1000, expr=None)
    
    ppl = []
    for result in results[0]:   
        
#         if result.distance < 1.5:
        
#           ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 
#            'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 'profile_picture', 
#            'city', 'state', 'country', 'zip_code', 'is_student', 'created_at', 'modified_at', 'user_id']
            
            try:
                sql = "select * from api_personalinformation where id = " + str(result.id) + ";"
                cursor.execute(sql)
                rows = cursor.fetchall()
                
                if len(rows[0][12]) == 0:
                    pp = rows[0][12]
                else:
                    pp = '/dev_asp_bucket/' + rows[0][12] 

                if len(rows) != 0:
                    ppl.append({'id' : rows[0][0], 
                                'first_name': rows[0][1],
                                'last_name': rows[0][2],
                                'personal_email': rows[0][3],
                                'personal_skills': rows[0][4],
                                'job_title': rows[0][5],
                                'headline': rows[0][6],
                                'bio': rows[0][7],
                                'office_phone': rows[0][8],
                                'personal_phone': rows[0][9],
                                'address_line_1': rows[0][10],
                                'address_line_2': rows[0][11],
                                'profile_picture': pp,
                                'city': rows[0][13], 
                                'state': rows[0][14], 
                                'country': rows[0][15], 
                                'zip_code': rows[0][16], 
                                'is_student': rows[0][17],
                                'created_at': rows[0][18], 
                                'modified_at': rows[0][19], 
                                'user_id': rows[0][20]})
                    
            except:
                print('query at query_ppl')
                connectdb()
    
    return ppl

In [37]:
# ppl = query_ppl('software developer')
# for i, j in enumerate(ppl):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
#     if i == 1:
#         break

## Comment

In [38]:
def update_cmt():
    
    if switch == True:
        TN = TABLE_NAME_5_
    else:
        TN = TABLE_NAME_5
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    oid = FieldSchema(name = "oid", dtype = DataType.INT64)
    cmt = FieldSchema(name = field_name_5, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, oid, cmt], description = "Comment embeddings with its object_id and id")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_5, index_params=index_param)
    
#   ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 'created_at_time', 
#    'updated_at_time', 'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']
    
    try:
        cursor.execute("Select id, object_id, user_comment from challenge_creator_comment")
        rows_pos = cursor.fetchall()
    except:
        print('query at update_cmt')
        connectdb()
    
    lst_pid, lst_oid, lst_cmt = [], [], []
    for i in rows_pos:
        
        if i[2] == None:
            continue
        
        lst_pid.append(i[0])  
        lst_oid.append(i[1])
        lst_cmt.append(preprocess(i[2]))
    
    if len(lst_cmt) != 0:
        sentence_embeddings = model.encode(lst_cmt)
        sentence_embeddings = normalize(sentence_embeddings)

        em = list(sentence_embeddings)
        mr = collection.insert([lst_pid, lst_oid, em])
        print('CMT updated on', TN)
    else:
        print("There is no Data")

In [39]:
# update_cmt()

In [40]:
def query_cmt(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = preprocess(q)
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_5)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_5_)
        collection.load()
    
    results = collection.search(query_embeddings, field_name_5, param=search_params, limit=1000, expr=None)
    
    cmt = []
    for result in results[0]: 
        
#         if result.distance < 1.5:
            
#           ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 'created_at_time', 
#            'updated_at_time', 'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']

#           ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
#            'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
#            'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
#            'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']  
            
            try:
                cmt_sql = "select * from challenge_creator_comment where id = " + str(result.id) + ";"
                cursor.execute(cmt_sql)
                rows_cmt = cursor.fetchall()
            except:
                print('query at query_cmt 1')
                connectdb()
            
            if len(rows_cmt) != 0:
                try:
                    sql = "select * from challenge_creator_challengestatement where id = {};".format(rows_cmt[0][1])
                    cursor.execute(sql)
                    rows = cursor.fetchall()
                except:
                    print('query at query_cmt 2')
                    connectdb()
                    
                try:
                    sql = "select profile_picture from api_personalinformation where user_id = {};".format(rows_cmt[0][7])
                    cursor.execute(sql)
                    rows_pp = cursor.fetchall()
                except:
                    print('query at query_cmt 3')
                    connectdb()

                if len(rows) != 0:
                    cmt.append({'id' : rows[0][0], 
                                'challenge_title': rows[0][1],
                                'challenge_description': rows[0][2],
                                'challenge_location': rows[0][3],
                                'challenge_state': rows[0][4],
                                'challenge_city': rows[0][5],
                                'skills': rows[0][6],
                                'status_type': rows[0][7],
                                'post_type': rows[0][8],
                                'is_active': rows[0][9],
                                'is_archieve': rows[0][10],
                                'is_student_required': rows[0][11],
                                'created_at': rows[0][12],
                                'challenge_end_date': rows[0][13],
                                'updated_at': rows[0][14],
                                'created_at_time': rows[0][15],
                                'updated_at_time': rows[0][16],
                                'company_name_id': rows[0][17],
                                'user_id': rows[0][18],
                                'comment_id': rows_cmt[0][0],
                                'user_comment': rows_cmt[0][2], 
                                'profile_picture': rows_pp[0][0]})

    return cmt

In [41]:
# cmt = query_cmt('This is a new')
# for i, j in enumerate(cmt):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
# #     if i == 1:
# #         break

---

## File Metadata

In [18]:
def fmeta(x):
    
    ### Download File ###
    
    fname = x.split('/')[-1]
    r = requests.get(x, stream=True)
    
    with open(fname, 'wb') as fd:
        fd.write(r.content)
    
    ############################################
    
    text = textract.process(fname) # x
    text = text.decode('utf8')
    
#     preprocess(fname)

    c = 0
    batch = ''
    batch_lst = []
    for i in text.split(' '):

        if c <= 63:
            batch = batch + ' ' + i
            c += 1

        else:
            batch_lst.append(str(preprocess(batch)))
            batch = i
            c = 1

    batch_lst.append(batch.strip())
    
    if os.path.exists(fname):
        os.remove(fname)
    else:
        print("The file does not exist")

    return batch_lst

In [19]:
# lst = fmeta(x = 'https://backend-alpha.thesciencepark.dev/api/user/download-image/dev_asp_bucket/documents/Sprint_7__8_-_Task_List.xlsx')
# lst = fmeta(x = 'potentially compromised credentials for SPARK MVP (id- spark-mvp-355400).pdf')

# for i in lst:
#     print(i)
#     print(type(i))
#     print('#'*54)


<class 'str'>
######################################################


---

In [43]:
def update_fsearch():
    
    if switch == True:
        TN = TABLE_NAME_6_
    else:
        TN = TABLE_NAME_6
    
    try:
        collection = Collection(name = TN)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    uid = FieldSchema(name = "uid", dtype = DataType.INT64)
    did = FieldSchema(name = "did", dtype = DataType.INT64)
    fl = FieldSchema(name = field_name_6, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, uid, did, fl], description = "File Search")
    collection = Collection(name = TN, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name = field_name_6, index_params = index_param)  
    
    try:
        cursor.execute("Select id, unique_id, document_id, mdata from api_filemeta")
        rows_pos = cursor.fetchall()
        
#         pid = primary_id, uid = unique_id (document), did = document_id (table_type)
        lst_pid, lst_uid, lst_did, lst_fl = [], [], [], []
        for i in rows_pos:
            
            if i[3] == None or i[3] == '':
                continue
                
            lst_pid.append(i[0])  
            lst_uid.append(i[1])
            lst_did.append(i[2])
            lst_fl.append(i[3])

        if len(lst_fl) != 0:
            sentence_embeddings = model.encode(lst_fl)
            sentence_embeddings = normalize(sentence_embeddings)

            em = list(sentence_embeddings)
            mr = collection.insert([lst_pid, lst_uid, lst_did, em])
            print('FSEARCH updated on', TN)
        else:
            print("There is no Data")
    
    except:
        print('query at update_fsearch')
        connectdb()

In [45]:
# update_fsearch()

FSEARCH updated on doc_Embds


In [ ]:
# tst = [1, 1, 2, 3, 5, 7, 8, 8, 8, 8]
# c = Counter(tst)
# print(c)

# sort = operator.itemgetter(1)
# sorted_tst = sorted(c.items(), key=sort, reverse=True)
# print(sorted_tst)

In [15]:
def query_fsearch(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    if switch == True:
        collection = Collection(TABLE_NAME_6)
        collection.load()
    else:
        collection = Collection(TABLE_NAME_6_)
        collection.load()
    
    results = collection.search(query_embeddings, field_name_6, param=search_params, limit=1000, expr=None)
    
    l = []
    for result in results[0]:
        if result.distance < 1.2:
            l.append(result.id)
    
    res = collection.query(expr = "id in {}".format(l), offset = 0, limit = 1000, 
                           output_fields = ["id", "uid", 'did'], consistency_level = "Strong")
    
#     print(res)
    
    uid_lst_0, uid_lst_1, uid_lst_2 = [], [], []
    for i in res:
        
        if i['did'] == 0:
            uid_lst_0.append(i['uid'])
        if i['did'] == 1:
            uid_lst_1.append(i['uid'])
        if i['did'] == 2:
            uid_lst_2.append(i['uid'])
#     print('='*50)
#     print(uid_lst_0)
#     print(uid_lst_1)
#     print(uid_lst_2)
    
    c0 = Counter(uid_lst_0)
    c1 = Counter(uid_lst_1)
    c2 = Counter(uid_lst_2)

    sort = operator.itemgetter(1)
    sorted0 = sorted(c0.items(), key=sort, reverse=True)
    sorted1 = sorted(c1.items(), key=sort, reverse=True)
    sorted2 = sorted(c2.items(), key=sort, reverse=True)
    
    # business_id     lab_profile_id
    
    personal_doc = []
    for i in sorted0:
        try:
            print(i[0])
            d0_sql = "select upload_documents, user_id from api_personaldocumentupload where id = " + str(i[0]) + ";"
            cursor.execute(d0_sql)
            rows_d0 = cursor.fetchall()
            
#             personal_doc.append(rows_d0)
            
            try:
                sql_p = "select first_name, last_name from api_personalinformation where id = " + str(rows_d0[0][-1]) + ";"
                cursor.execute(sql_p)
                rows_p = cursor.fetchall()

                fname_p = rows_p[0][0]
                lname_p = rows_p[0][1]
            except:
                print('query at query_fsearch 2')
                connectdb()
                fname_p = ''
                lname_p = ''
                
            personal_doc.append({'upload_documents': rows_d0[0][0],
                                'user_id': rows_d0[0][1],
                                'fname': fname_p,
                                'lname': lname_p})
        
        except:
            print('query at query_fsearch 1')
            connectdb()
        
    business_doc = []
    for i in sorted1:
        try:
            d1_sql = "select upload_documents, user_id, business_id from api_businessdocumentupload where id = " + str(i[0]) + ";"
            cursor.execute(d1_sql)
            rows_d1 = cursor.fetchall()
#             personal_doc.append(rows_d1)

            try:
                sql_b = "select company_name from api_businessinformation where id = " + str(rows_d1[0][-1]) + ";"
                cursor.execute(sql_b)
                rows_b = cursor.fetchall()

                company_name = rows_b[0][0]
            except:
                print('query at query_fsearch 4')
                connectdb()
                company_name = ''
                
            business_doc.append({'upload_documents': rows_d1[0][0],
                                'user_id': rows_d1[0][1],
                                'company_name': company_name})

        except:
            print('query at query_fsearch 3')
            connectdb()
        
    lab_doc = []
    for i in sorted2:
        try:
            d2_sql = "select upload_documents, user_id, lab_profile_id from lab_profile_labdocumentupload where id = " + str(i[0]) + ";"
            cursor.execute(d2_sql)
            rows_d2 = cursor.fetchall()
#             personal_doc.append(rows_d2)

            try:
                sql_d = "select company_name from lab_profile_labprofile where id = " + str(rows_d2[0][-1]) + ";"
                cursor.execute(sql_d)
                rows_d = cursor.fetchall()

                lab_name = rows_d[0][0]
            except:
                print('query at query_fsearch 6')
                connectdb()
                lab_name = ''
                
            personal_doc.append({'upload_documents': rows_d2[0][0],
                                'user_id': rows_d2[0][1],
                                'lab_name': lab_name})
    
        except:
            print('query at query_fsearch 5')
            connectdb()
            
#     print(personal_doc)
    
    return {'personal_doc': personal_doc, 'business_doc': business_doc, 'lab_doc': lab_doc}

In [16]:
# doc = query_fsearch('collabration is important because')

# for i, j in doc.items():
#     print(i, '==>') #, j)
#     print('='*50)
    
#     for l, k in enumerate(j):
#         print(l)
#         print('-'*50)
#         for m, n in k.items():
#             print(m, ':', n)
#     print('\n')
#     print('='*50)
#     print('='*50)
#     print('='*50)
#     print('\n')

[{'id': 2, 'uid': 33, 'did': 0}, {'id': 8, 'uid': 1, 'did': 1}]
[33]
[1]
[]
33
personal_doc ==>
0
--------------------------------------------------
upload_documents : documents/Baghel_Rajkumar.pdf
user_id : 2
fname : Manish
lname : Shaw




business_doc ==>
0
--------------------------------------------------
upload_documents : business/Challenge_Comment_and_Interaction_Template.docx
user_id : 2
company_name : Trapo india Pvt Ltd




lab_doc ==>






## Update

In [ ]:
def update():

    try:
        update_rcmd()
        print('RCMD UPDATED')
    except:
        print('Error in RCMD UPDATED')
    
    try:
        update_stdrcmd()
        print('STDRCMD UPDATED')
    except:
        print('Error in STDRCMD UPDATED')

    try:
        update_search()
        print('SEARCH UPDATED')
    except:
        print('Error in SEARCH UPDATED')

    try:
        update_biz()
        print('BIZ UPDATED')
    except:
        print('Error in BIZ UPDATED')

    try:
        update_ppl()
        print('PPL UPDATED')
    except:
        print('Error in PPL UPDATED')

    try:
        update_cmt()
        print('CMT UPDATED')
    except:
        print('Error in CMT UPDATED')
        
    try:
        update_fsearch()
        print('CMT FSEARCH')
    except:
        print('Error in FSEARCH UPDATED')
       
    global switch
    
    if switch == True:
        switch = False
    else:
        switch = True
        
    print('Switch is', switch)

In [ ]:
print(switch)

In [ ]:
# update()

---

In [ ]:
# def get_data_tst(q):
    
# #     args = request.args
# #     q = args.get('q')
    
#     if type(q) != str:
#         raise Exception("q must be a string")
        
#     search = query_search(q)
#     biz = query_biz(q)
#     ppl = query_ppl(q)
#     cmt = query_cmt(q)
    
#     return {'Search': search, 'Business': biz, 'People': ppl, 'Comment': cmt}

In [ ]:
# tst = get_data_tst('luigcduysgcuyh')
# print(1, '-'*25, '>')
# tst = get_data_tst('html')
# print(2, '-'*25, '>')
# tst = get_data_tst('manager')
# print(3, '-'*25, '>')
# tst = get_data_tst('AI')
# print(4, '-'*25, '>')
# tst = get_data_tst('Deep learning')
# print(5, '-'*25, '>')
# tst = get_data_tst('    hex')
# print(6, '-'*25, '>')
# tst = get_data_tst('data')
# print(7, '-'*25, '>')
# tst = get_data_tst('data management    ')
# print(8, '-'*25, '>')
# tst = get_data_tst('root')
# print(9, '-'*25, '>')
# tst = get_data_tst('Milan')
# print(10, '-'*25, '>')
# tst = get_data_tst('Saashq')
# print(11, '-'*25, '>')
# tst = get_data_tst('Canada')
# print(12, '-'*25, '>')
# tst = get_data_tst('India')
# print(13, '-'*25, '>')
# tst = get_data_tst('Alberta')
# print(14, '-'*25, '>')
# tst = get_data_tst('data management')
# print(15, '-'*25, '>')
# for i, j in enumerate(tst):
    
#     print(i, "-->")
#     print('='*25)
#     print(j.items())      
#     if i == 1
#         break

---

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/updt', methods=['GET'])
def updt():
    
    update()
        
#      try:
#         update_fsearch()
#         print('Fsearch UPDATED')
#     except:
#         print('Error in Fsearch UPDATED')
        
    return ('', 204)

#===========================================#
    
@app.route('/get_data', methods=['GET'])
@cross_origin()
def get_data():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    search = query_search(q)
    biz = query_biz(q)
    ppl = query_ppl(q)
    cmt = query_cmt(q)
    doc = query_fsearch(q)
    
    return {'Search': search, 'Business': biz, 'People': ppl, 'Comment': cmt, 'Document_Search': doc}
    
@app.route('/get_rcmd', methods=['GET'])
@cross_origin()
def get_rcmd():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    rcmd = query_rcmd(q)
    
    return rcmd

@app.route('/get_stdrcmd', methods=['GET'])
@cross_origin()
def get_stdrcmd():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    stdrcmd = query_stdrcmd(q)
    
    return stdrcmd

@app.route('/get_fmeta', methods=['GET'])
@cross_origin()
def get_fmeta():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    meta = fmeta(q)
    
    return meta

@app.route('/get_fsearch', methods=['GET'])
@cross_origin()
def get_fsearch():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    meta = query_fsearch(q)
    
    return meta

In [ ]:
from asgiref.wsgi import WsgiToAsgi

import asyncio
from hypercorn.config import Config
from hypercorn.asyncio import serve

from hypercorn.config import Config

config = Config()
config.bind = ["0.0.0.0:80"]

In [ ]:
if __name__ == '__main__':
    
    import logging
    logging.basicConfig(filename='error.log',level=logging.DEBUG)
    
    port = 80
    app.run(host='0.0.0.0', port=port, debug=True) # ssl_context=('cert.pem', 'key.pem')
#     asyncio.run(serve(app, config))

---

In [32]:
get_ipython().system('jupyter nbconvert text_search_engine_alberta_Copy1.ipynb --to python')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[NbConvertApp] Converting notebook text_search_engine_alberta_Copy1.ipynb to python
[NbConvertApp] Writing 68271 bytes to text_search_engine_alberta_Copy1.py


In [ ]:
# !pip install git+https://github.com/leukeleu/jupyter-linenumbers
# !jupyter nbextension install --py linenumbers --user
# !jupyter nbextension enable --py linenumbers --user